# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    # print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

256


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above

try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
query = "CREATE TABLE IF NOT EXISTS song_info_session"
query = query + "(session_id int, item_in_session int, artist varchar, song varchar, length float, PRIMARY KEY(session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)                   

In [9]:
query = "CREATE TABLE IF NOT EXISTS song_playlist_session"
query = query + "(user_id int, session_id int, artist varchar, song varchar, item_in_session int, first_name varchar, last_name varchar,  \
        PRIMARY KEY((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)  

In [10]:
query = "CREATE TABLE IF NOT EXISTS users_all_hands_against_his_own"
query = query + "(song varchar, user_id int, first_name varchar, last_name varchar, PRIMARY KEY(song, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)  

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query1 = "INSERT INTO song_info_session(session_id, item_in_session, artist, song, length)"
        query1 = query1 + " VALUES (%s, %s, %s, %s, %s)"
        query2 = "INSERT INTO song_playlist_session(user_id, session_id, artist, song, item_in_session, first_name, last_name)"
        query2 = query2 + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        query3 = "INSERT INTO users_all_hands_against_his_own(song, user_id, first_name, last_name)"
        query3 = query3 + " VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement
        session.execute(query1, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        session.execute(query2, (int(line[10]), int(line[8]), line[0], line[9], int(line[3]), line[1], line[4]))
        session.execute(query3, (line[9], int(line[10]), line[1], line[4]))

#### Check data inserted

In [12]:
## Verification of the data inserted in the song_info_session table

query = "select artist, song, length from song_info_session"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, '|', row.song, '|', row.length)

Bloodhound Gang | Uhn Tiss Uhn Tiss Uhn Tiss | 260.20526123046875
Habib KoitÃÂ© | Din Din Wo | 285.1521911621094
Sugarland | Just Might (Make Me Believe) | 247.77098083496094
A Hope For Home | Absolution: Of Flight and Failure | 388.38812255859375
Skillet | Monster (Album Version) | 178.02403259277344
Cat Stevens | Wild World | 200.20199584960938
Death Cab for Cutie | A Lack Of Color (Album Version) | 216.42404174804688
Tracy Gang Pussy | I Have A Wish | 221.33505249023438
Kanye West | Family Business | 278.8828125
Five Iron Frenzy | Canada | 236.09423828125
Morcheeba | The Sea | 347.7154541015625
The Avett Brothers | All My Mistakes (US4BW0700133) | 307.95709228515625
Creedence Clearwater Revival | Sweet Hitch-Hiker | 175.8036346435547
Kierra Sheard | Love Like Crazy (Extended Mix) | 299.0232238769531
Dave Darell | Children | 217.33831787109375
Klaus Badelt | Moonlight Serenade | 128.62649536132812
Frumpies | Fuck Kitty | 134.4779052734375
D-12 | Barbershop | 261.4068908691406
Kanye 

In [13]:
## Verification of the data inserted in the song_playlist_session table

query = "select artist, song, first_name, last_name from song_playlist_session"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, '|', row.song, '|', row.first_name, '|', row.last_name)

Rammstein | Adios | Isaac | Valdez
Gershon Kingsley | Pop Corn | Isaac | Valdez
The Rakes | Strasbourg | Isaac | Valdez
Jason Mraz & Colbie Caillat | Lucky (Album Version) | Kevin | Arellano
Chevelle | Dos (LP Version) | Adler | Barrera
Jorge Drexler | Salvapantallas | Adler | Barrera
The Mercury Program | Marianas | Adler | Barrera
Cat Stevens | Wild World | Kynnedi | Sanchez
Five Iron Frenzy | Canada | Tegan | Levine
Morcheeba | The Sea | Tegan | Levine
The Avett Brothers | All My Mistakes (US4BW0700133) | Tegan | Levine
Frumpies | Fuck Kitty | Aleena | Kirby
D-12 | Barbershop | Aleena | Kirby
Kanye West | Flashing Lights | Aleena | Kirby
Katy Perry | I Kissed A Girl | Aleena | Kirby
Drake / Kanye West / Lil Wayne / Eminem | Forever | Aleena | Kirby
Knights Of The Abyss | Dragon Pie | Aleena | Kirby
Nada Surf | Whose Authority | Aleena | Kirby
Blank & Jones | Beyond Time (Ambient Mix) | Aleena | Kirby
Broadcast 2000 | The View | Aleena | Kirby
Pixies | Where Is My Mind? | Aleena | Ki

In [14]:
## Verification of the data inserted in the users_all_hands_against_his_own table

query = "select first_name, last_name from users_all_hands_against_his_own"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.first_name, '|', row.last_name)

Lily | Koch
Sylvie | Cruz
Sara | Johnson
Sylvie | Cruz
Lily | Koch
Lily | Koch
Lily | Koch
Lily | Koch
Sara | Johnson
Lily | Koch
Lily | Koch
Lily | Koch
Lily | Koch
Samuel | Gonzalez
Lily | Koch
Sara | Johnson
Lily | Koch
Aiden | Hess
Lily | Koch
Lily | Koch
Connar | Moreno
Joseph | Gutierrez
Sara | Johnson
Lily | Koch
Lily | Koch
Celeste | Williams
Lily | Koch
Lily | Koch
Joseph | Gutierrez
Lily | Koch
Joseph | Gutierrez
Ayleen | Wise
Lily | Koch
Adler | Barrera
Sara | Johnson
Lily | Koch
Kaleb | Cook
Sara | Johnson
Lily | Koch
Sara | Johnson
Lily | Koch
Mohammad | Rodriguez
Lily | Koch
Sara | Johnson
Celeste | Williams
Sara | Johnson
Lily | Koch
Tegan | Levine
Lily | Koch
Lily | Koch
Lily | Koch
Ryan | Smith
Sara | Johnson
Sara | Johnson
Sara | Johnson
Aleena | Kirby
Lily | Koch
Lily | Koch
Lily | Koch
Ryan | Smith
Isaac | Valdez
Aleena | Kirby
Aleena | Kirby
Lily | Koch
Lily | Koch
Sara | Johnson
Sara | Johnson
Lily | Koch
Lily | Koch
Aleena | Kirby
Sara | Johnson
Makinley | Jones


### Results of the initial queries

In [15]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = "select artist, song, length from song_info_session WHERE session_id = 338 and item_in_session = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, '|', row.song, '|', row.length)                    

In [16]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = "select artist, song, first_name, last_name from song_playlist_session WHERE user_id = 10 and session_id = 182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, '|', row.song, '|', row.first_name, row.last_name)    

Down To The Bone | Keep On Keepin' On | Sylvie Cruz
Three Drives | Greece 2000 | Sylvie Cruz
Sebastien Tellier | Kilometer | Sylvie Cruz
Lonnie Gordon | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Sylvie Cruz


In [17]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "select first_name, last_name from users_all_hands_against_his_own WHERE song='All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)    

Sara Johnson


### Drop the tables before closing out the sessions

In [18]:
query1 = "DROP TABLE IF EXISTS song_info_session"
try:
    session.execute(query1)
except Exception as e:
    print(e) 

query2 = "DROP TABLE IF EXISTS song_playlist_session"
try:
    session.execute(query2)
except Exception as e:
    print(e) 
    
    
query3 = "DROP TABLE IF EXISTS users_all_hands_against_his_own"
try:
    session.execute(query3)
except Exception as e:
    print(e)  

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()